**Задача**


В этом задании вам предстоит ранжировать объявления для поиска.

Зачем вообще их ранжировать?
Поиск в Авито – трафикообразующая платформа для селлеров и незаменимый инструмент для покупателей. Если поисковая выдача будет нерелевантной, покупатели уйдут с площадки. Как и уйдут продавцы, если перестанут получать отклики и трафик на свои объявления.

Архитектура реальных поисковых систем обычно многостадийная: есть этап кандидатогенерации, работающий со всей базой объявлений, и есть этап реранкинга, поднимающий релевантных кандидатов выше в выдаче.

В задании предлагается разработать ML-модель для этапа реранкинга. Речь здесь идёт о небольшом числе объявлений, порядка десятков-сотен. Предполагается, что вы будете использовать классические методы машинного обучения, но если позволят время и силы применить что-то из мира DL (напр. двубашенные модели) – you're welcome.

https://ucarecdn.com/12f254c5-12b7-488e-8524-fa48c24d683e/

**Данные**

Коротко пройдемся по полям в наборе данных:

query_id и item_id – идентификаторы поискового запроса и объявления соответственно. Пара (query_id, item_id) представляет собой одну 'точку' в наборе данных 
query_text, item_title, item_description – доступные вам текстовые данные. Текст запроса, заголовок и описание объявления 
query_cat, query_mcat, query_loc – некоторые категории запроса и его локация 
item_cat_id, item_mcat_id, item_loc – аналогичные поля для объявления
price – цена, указанная в объявлении
item_query_click_conv – статистика, некоторая конверсия из показа в клик у этого айтема по этому запросу
item_contact – произошел ли у покупателя контакт с продавцом по этому айтему, колонка с таргетом. Естественно, она есть только в тренировочной выборке

In [3]:
import pandas as pd

train = pd.read_parquet('train-dset.parquet')
test = pd.read_parquet('test-dset-small.parquet')

In [5]:
test

,query_id,item_id,query_text,item_title,item_description,query_cat,query_mcat,query_loc,item_cat_id,item_mcat_id,item_loc,price,item_query_click_conv
0,55,7540855789,1 сентября,Воздушные и гелиевые шары на 1 сентября,ВОЗДУШНЫЕ ШАРЫ К 1 СЕНТЯБРЯ 🍂\n\n🎒 Друзья! Мы ...,114.0,63.0,637640.0,114,2301564,637640,120.0,-1.0
1,55,7506720336,1 сентября,1 сентября фотозона,🎈Фотозона из шаров на 1 сентября – создайте пр...,114.0,63.0,637640.0,114,2301564,637640,5000.0,-1.0
2,55,3110733862,1 сентября,Букет на 1 сентября из зефира,Букеты на 1 сентября \n\nФото 1. Букет Пионов ...,114.0,63.0,637640.0,114,1090077,637640,1200.0,-1.0
3,55,7587733901,1 сентября,Спектакль-пантомима на 1 сентября,Спектакль-пантомима на 1 сентября!\n\n👍🏼Идеаль...,114.0,63.0,637640.0,114,2301563,637640,0.0,-1.0
4,55,7552455685,1 сентября,Воздушные гелиевые шары с доставкой 1 сентября,Воздушные шары для оформления школ и классов н...,114.0,63.0,637640.0,114,2301564,637640,100.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
335343,824759,3584540577,утилизация бытовой техники,"Вывоз оргтехники, сотовых телефонов, телевизоров","Скупка на утилизацию не рабочей оргтехники, ко...",114.0,2094222.0,662210.0,114,2301618,662210,1.0,-1.0
335344,824759,4130130433,утилизация бытовой техники,Прием компьютерных и радиоэлектроных плат,Дopoгo пpинимaeм электрoнные плaты и кoмпьютеp...,114.0,2094222.0,662210.0,114,2301618,662210,10000.0,-1.0
335345,824759,2303947188,утилизация бытовой техники,Вывоз мусора. Демонтаж. Вывоз строительного му...,🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩\r\nВывоз мусора\r\nВывоз СТРО...,114.0,2094222.0,662210.0,114,2301617,662210,0.0,-1.0
335346,824759,1864686952,утилизация бытовой техники,Вывоз мусора,Вывоз мусора контейнерами 8 кубов\n Работаем в...,114.0,2094222.0,662210.0,114,2301617,662210,0.0,-1.0


In [6]:
train


,query_id,item_id,query_text,item_title,item_description,query_cat,query_mcat,query_loc,item_cat_id,item_mcat_id,item_loc,price,item_query_click_conv,item_contact
0,4,7349717282,ботинки детские zara 21,Ботинки детские Zara,Новые полуботинки фирмы Zara. \nразмеры 21 сте...,29.0,38.0,624480.0,29,2179540,638660,500.0,-1.0,0.0
1,4,7519735286,ботинки детские zara 21,Детские ботинки Zara унисекс,"Крутые ботинки, в отличном состоянии",29.0,38.0,624480.0,29,2179540,637640,250.0,-1.0,0.0
2,4,4384449104,ботинки детские zara 21,Ботинки детские zara,Челси димесезонные Zara \nВ идеальном состояни...,29.0,38.0,624480.0,29,2179540,623880,1500.0,-1.0,0.0
3,4,7283365509,ботинки детские zara 21,Детские ботиночки Zara 21 размер,АВИТО ДОСТАВКА .21 РАЗМЕР.,29.0,38.0,624480.0,29,2179540,628530,220.0,-1.0,0.0
4,4,4452768560,ботинки детские zara 21,Детские ботиночки zara размер 21,Детские ботинки Zara \nРазмер 21 - 13 см\nСост...,29.0,38.0,624480.0,29,2179540,637640,1648.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7781785,824764,2410860885,благоустройство участка под ключ,"Вспашка земли мотоблоком,газон под ключ","Всем читающим и заинтересованным, добрый день🤝...",114.0,63.0,107620.0,114,1178214,638120,1000.0,-1.0,0.0
7781786,824764,7467310892,благоустройство участка под ключ,Расчистка участка под ключ,РАСЧИСТКА УЧАСТКА ПОД КЛЮЧ \n\nРасчистка участ...,114.0,63.0,107620.0,114,1178215,639320,1500.0,-1.0,0.0
7781787,824764,7513725672,благоустройство участка под ключ,Благоустройство участка под ключ,👋🏻 Здравствуйте! Предлагаем услуги по благоуст...,114.0,63.0,107620.0,114,1178215,637800,10000.0,-1.0,0.0
7781788,824764,7550685038,благоустройство участка под ключ,Заезд на участок / Благоустройство,Благоустройство участков | Заезд под ключ\n\nВ...,114.0,63.0,107620.0,114,1178212,637900,35000.0,-1.0,0.0


#### Метрика
Качество ранжирования будет оцениваться по метрике NDCG@10, вики. На картинке ниже версия без нормализации: 

На практике, эффекта positional decay засчёт знаменателя можно добиться необязательно используя логарифм. 

В грейдере на степике метрика считается следующим образом, используя decay вида 0.97^position

### Решение

Разбил задачу на два этапа, т.к. хотел использовать значительную выборку. На первом этапе производятся:
- загрузка и сэмплирование
- генерация простых фичей
- расчет TF-IDF сходствах
- сохраняется промежуточный файл


In [ ]:
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")

# берем 40% 
TRAIN_SAMPLE_RATIO = 0.4  


def prepare_and_run_preprocessing(sample_ratio: float) -> None:

    print("Загрузка и начальная обработка")


    df_train = pd.read_parquet("train-dset.parquet")
    df_test = pd.read_parquet("test-dset-small.parquet")

    # Сэмплирование по query_id
    if sample_ratio < 1.0:
        unique_qids = df_train["query_id"].unique()
        sampled_qids = np.random.choice(
            unique_qids,
            size=int(len(unique_qids) * sample_ratio),
            replace=False
        )
        df_train = df_train[df_train["query_id"].isin(sampled_qids)].copy()

    df_train["is_train"] = 1
    df_test["is_train"] = 0
    df = pd.concat([df_train, df_test], axis=0, ignore_index=True)

    text_cols = ["query_text", "item_title", "item_description"]
    for col in text_cols:
        df[col] = df[col].fillna("").astype(str)

    df = df.assign(
        query_len=df["query_text"].str.len(),
        title_len=df["item_title"].str.len(),
        price_log=np.log1p(df["price"]),
        cat_match=(df["query_cat"] == df["item_cat_id"]).astype(int),
        mcat_match=(df["query_mcat"] == df["item_mcat_id"]).astype(int),
        loc_match=(df["query_loc"] == df["item_loc"]).astype(int),
    )

    # Обработка конверсии
    conv = df["item_query_click_conv"]
    mean_conv = conv[conv != -1].mean()
    df["is_conv_missing"] = (conv == -1).astype(int)
    df["item_query_click_conv"] = conv.replace(-1, mean_conv)

    # TF-IDF
    print("TF-IDF расчёт")
    df["item_full_text"] = df["item_title"] + " " + df["item_description"]

    tfidf_sims_series = df.groupby("query_id").apply(
        lambda group: cosine_similarity(
            TfidfVectorizer().fit_transform(
                [group["query_text"].iloc[0]] + group["item_full_text"].tolist()
            )
        )[0, 1:]
    )

    df["tfidf_sim"] = np.concatenate(tfidf_sims_series.values)
    df = df.drop(columns=["item_full_text"])

    # Сохраняем во временный файл (у меня ограничены мощности, и в два этапа делать удобнее)
    output_path = "data_with_tfidf.parquet"
    print(f">>> Сохраняем обработанные данные в файл: {output_path}")
    df.to_parquet(output_path, index=False)

    print("Файл готов")


if __name__ == "__main__":
    prepare_and_run_preprocessing(sample_ratio=TRAIN_SAMPLE_RATIO)


**Второй этап**

1. Вычисление семантического сходства между запросами и заголовками товаров с помощью модели SBERT

Используем: модель paraphrase-multilingual-MiniLM-L12-v2 - работает с русским языком, соотношение производительности к требованиям системы

2. Обучение модели LightGBM для ранжирования и затем делаем предсказания


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import torch
import warnings
warnings.filterwarnings('ignore')


SBERT_MODEL_NAME = 'paraphrase-multilingual-MiniLM-L12-v2'
# Уменьшаем batch_size
SBERT_BATCH_SIZE = 16 
# Обрабатываем данные большими кусками чтоб облегчить нагрузку на GPU
CHUNK_SIZE = 100000 

def add_sbert_similarity_mem_safe(df: pd.DataFrame) -> pd.DataFrame:
    print("Cемантическоt сходствj SBERT")
    try:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Используем {device} для SBERT")
        sbert_model = SentenceTransformer(SBERT_MODEL_NAME, device=device)
        
        unique_queries = df['query_text'].unique()
        q_embeddings = sbert_model.encode(
            unique_queries, convert_to_tensor=True, show_progress_bar=True,
            batch_size=SBERT_BATCH_SIZE
        )
        q_emb_map = {txt: emb for txt, emb in zip(unique_queries, q_embeddings)}
        query_embs = torch.stack([q_emb_map[txt] for txt in df['query_text']])

        print(f"Кодируем заголовки кусками по {CHUNK_SIZE}...")
        item_embs_list = []
        for i in tqdm(range(0, len(df), CHUNK_SIZE), desc="Item Chunks"):
            chunk_titles = df['item_title'][i:i+CHUNK_SIZE].tolist()
            chunk_embs = sbert_model.encode(
                chunk_titles, convert_to_tensor=True, show_progress_bar=False, 
                batch_size=SBERT_BATCH_SIZE
            )
            item_embs_list.append(chunk_embs)
            
            if device == 'cuda':
                torch.cuda.empty_cache()

        item_embs = torch.cat(item_embs_list)

        print("Расчет косинусной близости")
        sims = util.cos_sim(query_embs, item_embs)
        df['semantic_sim'] = np.diag(sims.cpu().numpy())

    except Exception as e:
        print("ОШИБКА SBERT.")
        df['semantic_sim'] = 0.0
        
    return df


def main():
    # Загрузка временных данных
    input_path = 'data_with_tfidf.parquet'
    print(f"Загрузка данных из '{input_path}'")
    df_full = pd.read_parquet(input_path)

    # SBERT
    df_full = add_sbert_similarity_mem_safe(df_full)
    
    #Обучение и предсказание
    print("Обучение модели")
    df_train = df_full[df_full['is_train'] == 1].copy()
    df_test = df_full[df_full['is_train'] == 0].copy()

    FEATURES = [
        'query_len', 'title_len', 'price_log', 'is_conv_missing', 
        'item_query_click_conv', 'cat_match', 'mcat_match', 'loc_match',
        'tfidf_sim', 'semantic_sim'
    ]
    TARGET = 'item_contact'
    
    X_train = df_train[FEATURES].fillna(0)
    y_train = df_train[TARGET]
    train_groups = df_train.groupby('query_id').size().to_numpy()
    X_test = df_test[FEATURES].fillna(0)
    
    lgb_params = {'objective': 'lambdarank', 'metric': 'ndcg', 'n_estimators': 1200, 'learning_rate': 0.05, 'random_state': 42, 'n_jobs': -1, 'verbose': -1, 'colsample_bytree': 0.8, 'subsample': 0.8}

    print("Обучаем LightGBM Ranker...")
    ranker = lgb.LGBMRanker(**lgb_params)
    ranker.fit(X=X_train, y=y_train, group=train_groups)
    
    print("Редактируем и сохраняем")
    df_test['pred_score'] = ranker.predict(X_test)
    submission = df_test.sort_values(by=['query_id', 'pred_score'], ascending=[True, False])
    submission[['query_id', 'item_id']].to_csv('solution.csv', index=False)
    
    print("Успех!")

if __name__ == '__main__':
    main()